In [1]:
using SwiftObjectStores

In [2]:
using ProgressMeter

In [393]:
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using WordStreams
using Distances
using StatsBase
using Training
using AdaGramCompat
using CorpusLoaders.Similarity

In [4]:
SwiftObjectStores.list(SwiftService(), "sensemodels")

Dict{Pair{Any,Any},Pair{Any,Any}} with 1 entry:
  Pair{Any,Any}("action","list_container_part") => Pair{Any,Any}("marker","")

In [18]:
function all_word_sense_vectors(ee::WordEmbedding, word)
    if haskey(ee.embedding, word)
        [ee.embedding[word]]
    else
        Vector{Float32}[]
    end
end


function all_word_sense_vectors(ee::WordSenseEmbedding, word)
    get(ee.embedding, word, Vector{Float32}[])
end

function all_word_sense_vectors(am::AdaGramCompat.AdaGramModel, word)
    if haskey(am.dict.word2id, word)
        wsv_mat = word_sense_vectors(am, word)
        [view(wsv_mat,:,ii) for ii in 1:size(wsv_mat,2)]
    else
        Vector{Float32}[]
    end
end

all_word_sense_vectors (generic function with 3 methods)

In [5]:
using JLD
#ee = load("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_300_i1.jld","ee");
eep = get_jld(SwiftService(), "sensemodels/plain/", "tokenised_lowercase_WestburyLab.wikicorp.201004_100_nosubsample.jld", "ee");


In [ ]:
using AdaGram
using AdaGramCompat
am = load("models/adagram/more_senses.adagram_model.jld", "am");

In [50]:
s_am = get_jld(SwiftService(), "sensemodels/adagram/", "semhuff_more_senses.adagram_model.jld", "am");

In [55]:
function get_sims(ee, wordpairs::Matrix{String})
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try
            wvs_1::Matrix{Float32} = hcat(all_word_sense_vectors(ee,wordpairs[ii,1])...)
            wvs_2::Matrix{Float32} = hcat(all_word_sense_vectors(ee,wordpairs[ii,2])...)
            sims[ii]=1-minimum(pairwise(CosineDist(), wvs_1,wvs_2))
        catch ex
            warn(ex)
            sims[ii]=NaN
        end
    end
    return sims
end


get_sims (generic function with 3 methods)

In [398]:
wordpairs, groundsim = load_wordsim353("./data/corpora/wordsim353/combined.csv");
wordpairs=lowercase.(wordpairs)

@show corspearman(groundsim,get_sims(ee,wordpairs))
@show corspearman(groundsim,get_sims(am,wordpairs))
@show corspearman(groundsim,get_sims(s_am,wordpairs));

corspearman(groundsim,get_sims(ee,wordpairs)) = 0.5454215969351677
corspearman(groundsim,get_sims(am,wordpairs)) = 0.5617419285275981
corspearman(groundsim,get_sims(s_am,wordpairs)) = 0.46129108146748254


0.46129108146748254

In [ ]:
reload("")

In [435]:
function window_context(index::Integer, context::Vector, window_size::Integer)
    window_lower_bound = max(index - window_size÷2, 1)
    window_upper_bound = min(index + window_size÷2 -1 , length(context))
    view(context, window_lower_bound:window_upper_bound)
end
Base.Test.@test window_context(5, split("1 2 3 4 6 7 8"), 2) == ["4", "6"]
Base.Test.@test window_context(5, split("1 2 3 4 6 7 8"), 4) == ["3" , "4", "6", "7"]
Base.Test.@test window_context(2, split("1 3 4 5"), 4) == ["1","3","4"]
Base.Test.@test window_context(4, split("1 2 3 5"), 4) == ["2","3","5"]
Base.Test.@test window_context(5, split("1 2 3 4 6 7 8"), typemax(Int)) == split("1 2 3 4 6 7 8")

Test Passed
  Expression: window_context(5,split("1 2 3 4 6 7 8"),typemax(Int)) == split("1 2 3 4 6 7 8")
   Evaluated: SubString{String}["1","2","3","4","6","7","8"] == SubString{String}["1","2","3","4","6","7","8"]

In [450]:
(scws_wordpairs, scws_groundsim, scws_contexts, scws_indexes) = CorpusLoaders.Similarity.load_scws("./data/corpora/SCWS/ratings.txt");
scws_windowed_contexts = ((iis, cont) -> window_context(iis, cont, 10)).(scws_indexes, scws_contexts);

In [437]:
function get_sims(ee::FixedWordSenseEmbedding, wordpairs::Matrix{String}, contexts::Matrix)
    sims = Vector{Float64}(size(wordpairs,1))
    for ii in 1:size(wordpairs,1)
        try 
            sense1 = WSD(ee, wordpairs[ii,1],contexts[ii,1];skip_oov=true)
            wv1 = ee.embedding[wordpairs[ii,1]][sense1]
            
            sense2 = WSD(ee, wordpairs[ii,2],contexts[ii,2];skip_oov=true)
            wv2 = ee.embedding[wordpairs[ii,2]][sense2]
            
            sims[ii]=1-cosine_dist(wv1,wv2)
        catch ex
            if typeof(ex)==KeyError
                warn(ex)
                sims[ii]=NaN
            else
                rethrow(ex)
            end
        end
    end
    return sims
end


get_sims (generic function with 3 methods)

In [438]:

local_sims = get_sims(ee, scws_wordpairs, scws_contexts)
corspearman(scws_groundsim, scws_local_sims)


global_syms = get_sims(am, scws_wordpairs)
corspearman(scws_groundsim, global_syms)


LoadError: LoadError: MethodError: no method matching get_sims(::WordEmbeddings.WordEmbedding, ::Array{String,2}, ::Array{Array{SubString,1},2})
Closest candidates are:
  get_sims(!Matched::WordEmbeddings.FixedWordSenseEmbedding, ::Array{String,2}, ::Array{T,2}) at In[437]:2
  get_sims(::Any, ::Array{String,2}) at In[55]:2
  get_sims(!Matched::WordEmbeddings.FixedWordSenseEmbedding, ::Array{String,2}) at In[8]:2
while loading In[438], in expression starting on line 2

In [442]:
function synthesize_embedding(am::AdaGramModel,context::AbstractVector, word::AbstractString)
    known_context = filter(c->haskey(am.dict.word2id, c), context)
    sum(all_word_sense_vectors(am, word).*disambiguate(am.vm, am.dict, word, known_context, false))
end

synthesize_embedding (generic function with 2 methods)

In [443]:
_lp=Dict()

Dict{Any,Any} with 0 entries

In [456]:
function normal_probs(logprobs::Vector)
    ret = copy(logprobs)
    max_lp = maximum(logprobs)
    ret.-=max_lp #Bring closer to zero
    map!(exp,ret)
    denom = sum(ret)
    ret./=denom
    ret
end


function synthesize_embedding2(ee,context::AbstractVector, word_or_phrase::AbstractString, smoothby=1)
    logprobs, wvs = get!(_lp, (ee, context, word_or_phrase)) do
        words = split(word_or_phrase, " ")
        wvs = vcat((all_word_sense_vectors(ee,w) for w in words)...)
        if length(wvs) == 0
                throw(KeyError("None of $words have embeddings"))
        end
        lps = [logprob_of_context(ee, context, wv; skip_oov=true, normalise_over_length=false) 
                 for wv in wvs]
        (lps, wvs)
    end
    slogprobs = logprobs./(smoothby*length(context))
    sum(normal_probs(slogprobs).*wvs)
end
    
    

"""
synthesize_embedding:: Embeddings, Context, Word -> WordVector
"""
    function get_sims_synth(ee, wordpairs::Matrix{String}, contexts::Matrix, smoothby=1)
    sims = Vector{Float64}(size(wordpairs,1))
    npairs = size(wordpairs,1)
        #@showprogress 
        for ii in 1:npairs
        try           
            wv1 = synthesize_embedding2(ee, contexts[ii,1],wordpairs[ii,1], smoothby)
            wv2 = synthesize_embedding2(ee, contexts[ii,2],wordpairs[ii,2], smoothby)
             
            sims[ii]=1-cosine_dist(wv1,wv2)
        catch ex
            if typeof(ex)==KeyError
                #warn(ex)
                sims[ii]=NaN
            else
                rethrow(ex)
            end
        end
    end
    return sims
end


get_sims_synth

In [457]:
sims = get_sims_synth(am, scws_wordpairs, scws_contexts, 0.5)
corspearman(scws_groundsim, sims)

0.6259432652457774

In [464]:
corspearman(scws_groundsim, get_sims_synth(am, scws_wordpairs, windowed_scws_contexts,1))

0.6288843132192203

In [465]:
corspearman(scws_groundsim, get_sims_synth(s_am, scws_wordpairs, scws_contexts, 1))

0.5081117677519068

In [466]:
corspearman(scws_groundsim, get_sims_synth(s_am, scws_wordpairs, windowed_scws_contexts,1))

0.5104268665100573

In [280]:
cor = corspearman(scws_groundsim, get_sims_synth(am, scws_wordpairs,scws_contexts, 2/e) )

0.645578469111277

In [241]:
cor = corspearman(scws_groundsim, get_sims_synth(am, scws_wordpairs,scws_contexts, 0.74))

0.6456029011434851

In [284]:
r=[]
for ii in 0.1:0.5:100
    cor = corspearman(scws_groundsim, get_sims_synth(s_am, scws_wordpairs,scws_contexts, ii))
    push!(r,(ii,cor))
    println(ii,"\t", cor)
    flush(STDOUT)
end

0.1	0.1901592119634672
0.6	0.21081064034680186
1.1	0.2232107602756684
1.6	0.2445733673287734
2.1	0.2699877876956494
2.6	0.2960437526239092
3.1	0.3217219371963249
3.6	0.34742446569301183
4.1	0.370551342297871
4.6	0.39131418889500985
5.1	0.4102833427032488
5.6	0.4267281787375052
6.1	0.4401471840086132
6.6	0.4513594828295088
7.1	0.4610529715462046
7.6	0.46950547391993275
8.1	0.4770093972868335
8.6	0.4833151411956547
9.1	0.4889772934047452
9.6	0.49392164704539115
10.1	0.4983587702052475
10.6	0.5023084371411161
11.1	0.505629120744869
11.6	0.5085784951025566
12.1	0.5112266993586264
12.6	0.5134565818625343
13.1	0.5152945664365541
13.6	0.5172089532852242
14.1	0.5190332708835231
14.6	0.5206072935779904
15.1	0.5218078181550769
15.6	0.5228658625079464
16.1	0.5238489865460638
16.6	0.524700315671062
17.1	0.5253796450367332
17.6	0.5259809458643119
18.1	0.5264965355897231
18.6	0.5270487704170446
19.1	0.5274677497494963
19.6	0.528000712833776
20.1	0.5284733834987514
20.6	0.5287276400759285
21.1	0.5289

LoadError: LoadError: InterruptException:
while loading In[284], in expression starting on line 2

In [230]:
using UnicodePlots

In [285]:
lineplot(first.(r), (x->x[2]).(r))



       ┌────────────────────────────────────────┐ 
   0.6 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⡠⠤⠒⠒⠋⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⠓⠒⠂│ 
       │⠀⠀⠀⠀⠀⡤⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⢀⡎⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⡜⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⢠⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⡜⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⢠⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⡸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⢀⠇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⡜⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
   0.1 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
       └────────────────────────────────────────┘ 
       0                                       60


In [235]:
lineplot(first.(r), (x->x[2]).(r))

        ┌────────────────────────────────────────┐ 
   0.65 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⡤⢄⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡔⠁⠀⠀⠀⠉⠓⠦⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡜⠀⠀⠀⠀⠀⠀⠀⠀⠈⠑⠦⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢰⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠒⠤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡎⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠒⠦⣄⡀⠀⠀⠀⠀⠀⠀⠀│ 
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠙⠒⠦⢤⣀⡀⠀│ 
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⡜⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠙│ 
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
        │⠀⠀⠀⠀⠀⠀⠀⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
        │⠀⠀⠀⠀⠀⠀⠀⠀⡸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
        │⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
        │⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
        │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
   0.61 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ 
        └────────────────────────────────────────┘ 
        0                                        2


In [256]:
@show i=findmax((x->x[2]).(r))
r[i[2]]

i = findmax(((x->begin  # In[256], line 1:
                x[2]
            end)).(r)) = (0.5295573706348757,16)


(0.25,0.5295573706348757)

In [ ]:
# 0.5342692139056353     GlobalMin   plain/tokenised_lowercase_WestburyLab.wikicorp.201004_100_nosubsample.jld
# 0.48861817784625977    GlobalMin   adagram/more_senses.adagram_model.jld
# 0.45416228327202474    GlobalMin   sensemodels/adagram/semhuff_more_senses.adagram_model.jld"
# 0.523055629491102      GlobalMin   sensemodels/adagram/semhuff_more_senses.adagram_model.jld"
# 0.524743130161508      LocalSynth   greedy/tokenised_lowercase_WestburyLab.wikicorp.201004_300_i1.jl
# 0.6452187057383217     LocalSynth   adagram/more_senses.adagram_model.jld